In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from textwrap import dedent     # tap(들여쓰기)에 상관없이 쓸 수 있게.
from pprint import pprint

In [3]:
from langchain_community.tools import TavilySearchResults

query = "스테이크와 어울리는 와인을 추천해 주세요"

from langchain_community.tools import DuckDuckGoSearchResults

web_search = DuckDuckGoSearchResults(max_results=2)

# web_search = TavilySearchResults(max_results=2)     # max_result의 갯수까지만 문서를 가져오겠다.

search_results = web_search.invoke(query)           # TavilySearch도 검색 결과기 때문에 invoke() 사용

for result in search_results:
    print(result)
    print('-'*50)

c:\Users\scv31\AppData\Local\pypoetry\Cache\virtualenvs\langgraph-works-VZ1_LVEb-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\scv31\AppData\Local\pypoetry\Cache\virtualenvs\langgraph-works-VZ1_LVEb-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


s
--------------------------------------------------
n
--------------------------------------------------
i
--------------------------------------------------
p
--------------------------------------------------
p
--------------------------------------------------
e
--------------------------------------------------
t
--------------------------------------------------
:
--------------------------------------------------
 
--------------------------------------------------
S
--------------------------------------------------
e
--------------------------------------------------
p
--------------------------------------------------
 
--------------------------------------------------
2
--------------------------------------------------
4
--------------------------------------------------
,
--------------------------------------------------
 
--------------------------------------------------
2
--------------------------------------------------
0
--------------------------------------------

In [4]:
print(type(web_search))             # 타입반환

<class 'langchain_community.tools.ddg_search.tool.DuckDuckGoSearchResults'>


In [5]:
print(web_search.name)              # 풀네임

duckduckgo_results_json


In [6]:
print(web_search.description)       # llm을 위해 설명하는 것

A wrapper around Duck Duck Go Search. Useful for when you need to answer questions about current events. Input should be a search query. Output is a JSON string array of the query results


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools=[web_search])

In [8]:
# 간단한 쿼리문 입력
query = "안녕하세요"
ai_msg = llm_with_tools.invoke(query)
pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)
pprint('-'*100)

pprint(ai_msg.tool_calls)           # llm이 tool을 호출할지 말지를 판단할 수 있음. 
pprint('-'*100)

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 87, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'finish_reason': 'stop', 'logprobs': None}, id='run--62175b6f-a802-4952-97be-b611dac740ab-0', usage_metadata={'input_tokens': 87, 'output_tokens': 11, 'total_tokens': 98})
'----------------------------------------------------------------------------------------------------'
'안녕하세요! 어떻게 도와드릴까요?'
'----------------------------------------------------------------------------------------------------'
[]
'----------------------------------------------------------------------------------------------------'


In [9]:
# 복잡한 쿼리를 입력
query = "스테이크와 어울리는 와인을 추천해 주세요."
ai_msg = llm_with_tools.invoke(query)
pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)              # 컨텐츠를 사용하지 않을 때는 tool을 사용한 경우임. 따라서 llm이 tool을 쓰는데 content를 반환하면 잘못 설계된것
pprint('-'*100)
# tool을 가진 llm에게는 어떤 tool을 호출할지 맡기는 것을 중요하게 봄. [내부에는 복수의 tool 호출 가능하다.]
pprint(ai_msg.tool_calls)
pprint('-'*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uNR3IU8OFuL2iBv0g5j0hRyq', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'duckduckgo_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 98, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5e192eab-ecb4-4cc0-a134-f2022a61be12-0', tool_calls=[{'name': 'duckduckgo_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_uNR3IU8OFuL2iBv0g5j0hRyq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 98, 'output_tokens': 27, 'total_tokens': 125})
'------------------------------------------------------

In [10]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'duckduckgo_results_json',
 'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_uNR3IU8OFuL2iBv0g5j0hRyq',
 'type': 'tool_call'}

In [11]:
# tool call에 대해서 어떻게 생각 하는지 찍어 보기 위함.
tool_message = web_search.invoke(tool_call)

print(tool_message)

c:\Users\scv31\AppData\Local\pypoetry\Cache\virtualenvs\langgraph-works-VZ1_LVEb-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\scv31\AppData\Local\pypoetry\Cache\virtualenvs\langgraph-works-VZ1_LVEb-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


content='snippet: Telemedicine is being used in rural areas, though there is variation in use by demographics suggesting opportunities to encourage greater …, title: Telemedicine in Rural Communities: Patterns of Use and User ..., link: https://www.elevancehealth.com/content/dam/elevance-health/articles/ppi_assets/reports/PPI_Telemedicine_in_Rural_Areas.pdf, snippet: Telehealth was introduced as a temporary measure but is being considered as a more permanent form of healthcare, particularly in …, title: Telehealth in Response to the Rural Health Disparity - PMC, link: https://pmc.ncbi.nlm.nih.gov/articles/PMC9392842/, snippet: Oct 21, 2024 · COVID-19 has significantly impacted rural healthcare, leading to the implementation of innovative solutions like …, title: Telehealth and digital tools enhancing healthcare access in rural ..., link: https://link.springer.com/article/10.1186/s12982-024-00271-1, snippet: Apr 1, 2023 · The primary objective of this paper is to understand telemedicine

In [12]:
pprint(tool_message.content)

('snippet: Telemedicine is being used in rural areas, though there is '
 'variation in use by demographics suggesting opportunities to encourage '
 'greater …, title: Telemedicine in Rural Communities: Patterns of Use and '
 'User ..., link: '
 'https://www.elevancehealth.com/content/dam/elevance-health/articles/ppi_assets/reports/PPI_Telemedicine_in_Rural_Areas.pdf, '
 'snippet: Telehealth was introduced as a temporary measure but is being '
 'considered as a more permanent form of healthcare, particularly in …, title: '
 'Telehealth in Response to the Rural Health Disparity - PMC, link: '
 'https://pmc.ncbi.nlm.nih.gov/articles/PMC9392842/, snippet: Oct 21, 2024 · '
 'COVID-19 has significantly impacted rural healthcare, leading to the '
 'implementation of innovative solutions like …, title: Telehealth and digital '
 'tools enhancing healthcare access in rural ..., link: '
 'https://link.springer.com/article/10.1186/s12982-024-00271-1, snippet: Apr '
 '1, 2023 · The primary objectiv

In [26]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

today= datetime.today().strftime("%Y-%m-%d")
# prompt 생성
prompt = ChatPromptTemplate([
    ("system", f"You are helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder","{messages}")             # tools의 호출에 대한 정보 로드
])
# llm과 tool 가져오기
llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tools = llm.bind_tools(tools=[web_search])
# llm chain 생성
llm_chain = prompt | llm_with_tools
# 함수를 체인화
@chain
def web_search_chain(user_input: str):
    input_ = {"user_input":user_input}
    ai_msg = llm_chain.invoke(input_)
    # ai의 메세지가 여러개 올 수 있기에 batch사용
    tool_msgs = web_search.batch(ai_msg.tool_calls)
    print("tool_msgs: \n", tool_msgs)
    print('-'*100)
    # 결과 반환
    # **변수 -> dictionary로 풀어서 사용 되기 위해 (이후 인자가 dictionary 형태로 오기때문에 unpack 문법을 사용함. )
    return llm_chain.invoke({**input_, "message":[ai_msg, *tool_msgs]})          # message는 placeholder의 (key, value)

response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 원화 가격은 얼마인가요?")
pprint(response.content)

c:\Users\scv31\AppData\Local\pypoetry\Cache\virtualenvs\langgraph-works-VZ1_LVEb-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\scv31\AppData\Local\pypoetry\Cache\virtualenvs\langgraph-works-VZ1_LVEb-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


tool_msgs: 
 [ToolMessage(content='snippet: Founded in 1743, Moët & Chandon celebrates life’s memorable moments with a range of unique champagnes for every occasion. …, title: 샴페인 Moët & Chandon을 만나보세요 | Moët & Chandon, link: https://www.moet.com/kr-kr, snippet: Oct 15, 2022 · 모엣샹동 임페리얼 시음 후기였습니다. 2만 원대에 나쁘지 않은 화이트와인을 생각하시고 거기에다가 뽀글뽀글 탄산을 더한 딱 그 맛입니다. 일반적인 화이트와인에 비해 가격대가 많이 비싼데 그만큼의 …, title: 모엣샹동 샴페인 임페리얼 가격 및 시음 솔직 후기 - 네이버 블로그, link: https://m.blog.naver.com/kiddy28/222901030498, snippet: Sep 16, 2023 · 모엣샹동 넥타 시리즈는 파인애플, 망고, 패션후르츠 등 열대과일의 향을 느낄수 있도록 만든 제품으로 상당히 달콤한 맛이 특징인 제품군입니다. 스위트제품과 로제 제품이 있으며 두제품 모두 도사쥐가 …, title: 모엣 샹동 임페리얼 종류 및 가격 총정리, link: https://outdoor.toesoon.com/entry/모엣-샹동-임페리얼-종류-및-가격-총정리, snippet: Dec 9, 2024 · 모엣샹동 샴페인, 어디서 구매할 수 있을까요? 모엣샹동 샴페인은 백화점, 대형 마트, 와인 전문점 등에서 쉽게 구매할 수 있습니다. 온라인 쇼핑몰을 이용하면 더욱 편리하게 구매할 수 있으며, 다양한 할인 …, title: 모엣샹동 샴페인, 어떻게 즐기고 어디서 구매할까요? 완벽 가이드, link: https://wineries.kr/contents/모엣샹동-샴페인-어떻게-즐기고-어디서-구매할까요-완벽/', name='duckduckgo_results_json

In [14]:
# 사용자 커스텀 도구 정의하기
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

@tool       # 커스텀 tool 생성 어노테이션
def search_web(query: str) -> str:
    """Searches the internet for information that does not exist in the database or 
    for the latest information."""
    
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    
    formatted_docs = "\n---\n".join([
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in docs
            ])

    if len(formatted_docs) > 0:
        return formatted_docs

    return "관련 정보를 찾을 수 없습니다."

In [ ]:
query = "스테이크와 어울리는 와인을 추천해 주세요."
search_result = search_web.invoke(query)
print(search_result)

In [ ]:
llm_with_tools = llm.bind_tools(tools=[search_web])

query = "스테이크와 어울리는 와인을 추천해 주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)              # 결과가 도출 안 될 수 있음. 웹 검색을 한 정보가 도출 될 때는 빈 값으로 나오게 됨.
pprint('-'*100)

pprint(ai_msg.tool_calls)           # llm이 tool을 호출할지 말지를 판단할 수 있음. 
pprint('-'*100)

In [ ]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

In [ ]:
import re
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

In [18]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

In [ ]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

# Chroma 인덱스 생성
menu_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


검색 결과: 2개
메뉴 번호: 1
메뉴 이름: 시그니처 스테이크

메뉴 번호: 8
메뉴 이름: 안심 스테이크 샐러드



In [ ]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


검색 결과: 2개
메뉴 번호: 10
메뉴 이름: 그랜지 2016

메뉴 번호: 9
메뉴 이름: 샤토 디켐 2015



In [16]:
# 벡터 저장소 로드
menu_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [19]:
@tool
def search_menu(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) >0: return docs
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

In [20]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# 벡터 저장소 로드
wine_db = Chroma(
   embedding_function=embeddings_model,   
   collection_name="restaurant_wine",
   persist_directory="./chroma_db",
)

@tool
def search_wine(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [21]:
llm_with_tools = llm.bind_tools(tools=[search_menu, search_wine])
query="시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)              # 결과가 도출 안 될 수 있음. 웹 검색을 한 정보가 도출 될 때는 빈 값으로 나오게 됨.
pprint('-'*100)

pprint(ai_msg.tool_calls)           # llm이 tool을 호출할지 말지를 판단할 수 있음. 
pprint('-'*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6YlHYhZpjqzctqaUdGr2rkSm', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'search_menu'}, 'type': 'function'}, {'id': 'call_IYwY8l9QUv4KpfxbvTh9KrNm', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'search_wine'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4a93fb50-fbeb-4ece-aaad-92c0796c4970-0', tool_calls=[{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_6YlHYhZpjqzctqaUdGr2rkSm', 'type': 'tool_call'}, {'name': 'search_wine', 'args': {'query

In [22]:
tools = [search_web, search_wine, search_menu]
for tool in tools:
    print(tool.name)

search_web
search_wine
search_menu


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 3개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)       

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
# response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")
response = restaurant_menu_chain.invoke("전세계에서 가장 비싼 스테이크에 대해 알려 주세요.")

# 응답 출력 
print(response.content)

search_web: 
{'name': 'search_web', 'args': {'query': "world's most expensive steak 2025"}, 'id': 'call_m5w2XQQLxygh7RX0ER3JehcQ', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------


TypeError: string indices must be integers, not 'str'

In [ ]:
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려 주세요.")

In [ ]:
# few shot
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_menu", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. 주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_wine", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. 2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, 주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. 특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오, 그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]


system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,  #*examples는 리스트의 각 항목을 펼쳐서(unpack) 넣어주는 문법입니다.
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    # 에이전트가 도구를 호출한 흔적, 즉 생각 과정 + 도구 호출 요청 + 도구 응답 등을 여기에 넣습니다.
])

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [search_web, search_menu, search_wine]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."

agent_response = agent_executor.invoke({"input": query})